In [12]:
import pandas as pd
import os
from datetime import datetime, timedelta
from tqdm import tqdm
import gc
import sys
import matplotlib.pyplot as plt
import MetaTrader5 as mt5

path_root = "D:/Documentos/Erik/TDR/TDR-Forex/"
sys.path.append(path_root)
from utils.mt5 import *

## Posicions

In [13]:
# Retorna una taula amb les posicions obertes
posicions = get_positions_df()
posicions

,ticket,time,time_msc,time_update,time_update_msc,type,magic,identifier,reason,volume,price_open,sl,tp,price_current,swap,profit,symbol,comment,external_id
0,4434485425,2025-08-27 12:55:01,2025-08-27 12:55:01.969,2025-08-27 12:55:01,2025-08-27 12:55:01.969,1,1006,4434485425,3,0.1,1.159,1.18399,1.14899,1.15843,0.0,5.7,EURUSD,Mitjanes,


## Deals

#### `type`: Indica el tipus d’operació o transacció:
- **0**: Buy  
- **1**: Sell  
- **2**: Balance  

#### `entry`: Especifica com s’ha introduït o tancat la posició:
- **0**: In (Obertura)  
- **1**: Out (Tancament)  
- **2**: In/Out (Modificació parcial, increment o reducció)  

#### `reason`: Mostra la raó per la qual s’ha executat l’operació:
- **0**: Client (Execució manual)  
- **1**: Expert (Expert Advisor)  
- **2**: Dealer (Broker)  
- **3**: Signal (Subscripció a senyal)  
- **4**: Stop Loss (SL activat)  
- **5**: Take Profit (TP activat)  
- **6**: Stop Out (Tancament forçat per marge)  
- **7**: Roll Over (Ajustaments com swaps o contractes)  
- **8**: SL/TP (Altres execucions automàtiques)  


In [4]:
# Retorna una operacions realitzades
deals = get_deals_df()
deals.tail(2)

,ticket,order,time,time_msc,type,entry,magic,position_id,reason,volume,price,commission,swap,profit,fee,symbol,comment,external_id
9,52975712813,53026135794,2025-08-20 19:12:09,2025-08-20 19:12:09.403,0,1,2006,53026135782,3,0.01,1.16546,0.0,0.0,-0.11,0.0,EURUSD,,
10,52975712820,53026135803,2025-08-20 19:12:09,2025-08-20 19:12:09.481,1,1,2006,53026135774,3,0.01,1.16534,0.0,0.0,-0.10,0.0,EURUSD,,


### Obrir i modificar operacions

In [5]:
# Obrir una posició llarga. Retorna l'identificador de l'operació
argSymbol = "EURUSD"; argLotSize = 0.10; argSlippage = 10; argMagicNumber = 1974; argComment = "Twin Hours EMA"; argSLPips = 10; argTPPips =25
ticket = OpenBuyOrder(argSymbol, argLotSize, argSlippage, argMagicNumber, argComment, argSLPips, argTPPips)
ticket

Obrint ordre BUY: EURUSD 0.1 lots
Preu: 1.15857
SL: 1.15757 (10 pips)
TP: 1.16107 (25 pips)
Ordre executada correctament!
   Order: 53063709248, Deal: 53012386193
   SL: 1.15757
   TP: 1.16107


53063709248

In [6]:
# Obrir una posició curta. Retorna l'identificador de l'operació
argSymbol = "EURUSD"; argLotSize = 0.10; argSlippage = 10; argMagicNumber = 1974; argComment = "Twin Hours EMA"; argSLPips = 10; argTPPips =25
ticket = OpenSellOrder(argSymbol, argLotSize, argSlippage, argMagicNumber, argComment, argSLPips, argTPPips)
ticket

Obrint ordre SELL: EURUSD 0.1 lots
Preu: 1.15843
SL: 1.15943 (10 pips)
TP: 1.15593 (25 pips)
Ordre de venda executada correctament!
   Order: 53063709347, Deal: 53012386284
   SL: 1.15943
   TP: 1.15593


53063709347

In [7]:
posicions = get_positions_df()
posicions

,ticket,time,time_msc,time_update,time_update_msc,type,magic,identifier,reason,volume,price_open,sl,tp,price_current,swap,profit,symbol,comment,external_id
0,53063709248,2025-08-27 11:50:07,2025-08-27 11:50:07.080,2025-08-27 11:50:07,2025-08-27 11:50:07.080,0,1974,53063709248,3,0.1,1.15855,1.15757,1.16107,1.15845,0.0,-0.86,EURUSD,Twin Hours EMA,
1,53063709347,2025-08-27 11:50:08,2025-08-27 11:50:08.278,2025-08-27 11:50:08,2025-08-27 11:50:08.278,1,1974,53063709347,3,0.1,1.15843,1.15943,1.15593,1.15857,0.0,-1.21,EURUSD,Twin Hours EMA,


In [14]:
# Afegim SL i TP a una posició oberta
Modify_SL_and_TP(4434485425, 1.1615, 1.1580)

Change SL-TP 4434485425 successfully


In [9]:
# Tanquem una posició pel seu indicador
CloseOrderByTicket(ticket, 10)

Ticket 53063709347 closed successfully


In [10]:
# Tanquem totes les posicions obertes d'una divisa i un identificador
CloseAllOrders("EURUSD", 1974, 10)

Closed successfully: retcode=10009 - deal=53012387332 - order=53063710432
